    000_16_230114_141_subway_transer_23
    
    data : 023_신분당선 주소데이터_01
     save : subway_info_23.csv
     display : subway_info_23.html

In [39]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [40]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [41]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [42]:
#path = 'c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_addr/001_수도권1호선_주소데이터_20221122.csv'

subway_info_df = pd.read_csv('c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_addr/023_신분당선 주소데이터_01.csv')
subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소
0,국가철도,신분당선,강남,서울특별시 강남구 역삼동 858,서울특별시 강남구 강남대로 396
1,국가철도,신분당선,양재(서초구청),서울특별시 서초구 서초동 1366-9,서울특별시 서초구 남부순환로 2585
2,국가철도,신분당선,양재시민의숲(매헌),서울특별시 서초구 양재동 233-2,서울특별시 서초구 매헌로 116
3,국가철도,신분당선,청계산입구,서울특별시 서초구 신원동 270-25,서울특별시 서초구 청계산로 179
4,국가철도,신분당선,판교(판교테크노밸리),경기도 성남시 분당구 백현동 19-2,경기도 성남시 분당구 판교역로 160


In [43]:
subway_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   철도운영기관명  13 non-null     object
 1   선명       13 non-null     object
 2   역명       13 non-null     object
 3   지번주소     13 non-null     object
 4   도로명주소    13 non-null     object
dtypes: object(5)
memory usage: 648.0+ bytes


In [44]:
# subway_info_df['주소'] = subway_info_df['주소'].astype(str)

In [45]:
#5  university   # 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)

    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

슝~


In [46]:
#5  university .. 주소 기반 위도,경도 추가
for _, row in subway_info_df.iterrows():
    try:
        x, y = get_adress_x_y(row["도로명주소"])
     
        subway_info_df.loc[_, "x"] = x
        subway_info_df.loc[_, "y"] = y
    except:
        print(row["도로명주소"] + '은(는) 검색에 실패했습니다')

subway_info_df.head()


,철도운영기관명,선명,역명,지번주소,도로명주소,x,y
0,국가철도,신분당선,강남,서울특별시 강남구 역삼동 858,서울특별시 강남구 강남대로 396,958307.262586959,1944427.77368496
1,국가철도,신분당선,양재(서초구청),서울특별시 서초구 서초동 1366-9,서울특별시 서초구 남부순환로 2585,958873.936120214,1942825.95952766
2,국가철도,신분당선,양재시민의숲(매헌),서울특별시 서초구 양재동 233-2,서울특별시 서초구 매헌로 116,959180.129341039,1941253.91615499
3,국가철도,신분당선,청계산입구,서울특별시 서초구 신원동 270-25,서울특별시 서초구 청계산로 179,960605.854433113,1938892.26145651
4,국가철도,신분당선,판교(판교테크노밸리),경기도 성남시 분당구 백현동 19-2,경기도 성남시 분당구 판교역로 160,965554.33536144,1932922.48207143


In [47]:
#5  university  ..# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')


슝~


In [48]:
#5  university  .. 좌표계가 UTM-K이므로 WSG84좌표계로 변환
for _, row in subway_info_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        subway_info_df.loc[_, "lng"] = x
        subway_info_df.loc[_, "lat"] = y

    except:
        pass
subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소,x,y,lng,lat
0,국가철도,신분당선,강남,서울특별시 강남구 역삼동 858,서울특별시 강남구 강남대로 396,958307.262586959,1944427.77368496,127.028323,37.498170
1,국가철도,신분당선,양재(서초구청),서울특별시 서초구 서초동 1366-9,서울특별시 서초구 남부순환로 2585,958873.936120214,1942825.95952766,127.034823,37.483758
2,국가철도,신분당선,양재시민의숲(매헌),서울특별시 서초구 양재동 233-2,서울특별시 서초구 매헌로 116,959180.129341039,1941253.91615499,127.038373,37.469602
3,국가철도,신분당선,청계산입구,서울특별시 서초구 신원동 270-25,서울특별시 서초구 청계산로 179,960605.854433113,1938892.26145651,127.054622,37.448377
4,국가철도,신분당선,판교(판교테크노밸리),경기도 성남시 분당구 백현동 19-2,경기도 성남시 분당구 판교역로 160,965554.33536144,1932922.48207143,127.110846,37.394766


In [49]:
#5  university  .. 시각화 .. 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

슝~


In [50]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in subway_info_df.iterrows():
    try:
        folium.Marker(
            location=[subway_info_df.loc[idx, "lat"], subway_info_df.loc[idx, "lng"]],
            popup=subway_info_df.loc[idx, "역명"],
            icon=folium.Icon(color="cadetblue", icon="flag")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [51]:
#5 seoul university 
map

In [52]:
map.save("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/003_map_xml/subway_info_23.html")


In [53]:
#apt_df.to_excel("C:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/data/apt_cn_sung.xls", index = False)
# subway_info_df.to_csv("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_info_02.csv", index = False)
subway_info_df.to_csv("subway_info_23.csv", index = False)
